In [56]:
# import boto3
# import os

# from sagemaker import get_execution_role, Session

# # Initialize a SageMaker session
# sagemaker_session = Session()



# role = "arn:aws:iam::016114370410:role/tf-binding-sites"

# # prefix = "inference/output/AR-22Rv1-greater-2024-08-09-10-18-34/"
# prefix = "inference/output/AR-22Rv1-greater-equal-2024-08-09-10-27-48/"

# local_dir = "/Users/wejarrard/projects/tf-binding/data/jsonl_output"


# # Initialize the S3 client
# s3 = boto3.client('s3')

# # Specify your S3 bucket name
# bucket_name = sagemaker_session.default_bucket()


# # Function to download files from an S3 bucket
# def download_from_s3(bucket_name, prefix, local_dir):
#     if not os.path.exists(local_dir):
#         os.makedirs(local_dir)

#     # List objects within the specified prefix
#     response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

#     # Iterate over each object
#     for obj in response.get('Contents', []):
#         key = obj['Key']
#         # Skip if it's a folder
#         if key.endswith('/'):
#             continue
#         local_path = os.path.join(local_dir, os.path.basename(key))
        
#         # Download the file
#         s3.download_file(bucket_name, key, local_path)
#         print(f'Downloaded {key} to {local_path}')

# # Bucket name extracted from the S3 URI
# bucket_name = "tf-binding-sites"

# # delete the files in local dir
# for file in os.listdir(local_dir):
#     os.remove(os.path.join(local_dir, file))
    

# # Download the files
# download_from_s3(bucket_name, prefix, "/Users/wejarrard/projects/tf-binding/data/jsonl_output")

Downloaded inference/output/AR-22Rv1-greater-equal-2024-08-09-10-27-48/dataset_1.jsonl.gz.out to /Users/wejarrard/projects/tf-binding/data/jsonl_output/dataset_1.jsonl.gz.out
Downloaded inference/output/AR-22Rv1-greater-equal-2024-08-09-10-27-48/dataset_10.jsonl.gz.out to /Users/wejarrard/projects/tf-binding/data/jsonl_output/dataset_10.jsonl.gz.out
Downloaded inference/output/AR-22Rv1-greater-equal-2024-08-09-10-27-48/dataset_100.jsonl.gz.out to /Users/wejarrard/projects/tf-binding/data/jsonl_output/dataset_100.jsonl.gz.out
Downloaded inference/output/AR-22Rv1-greater-equal-2024-08-09-10-27-48/dataset_101.jsonl.gz.out to /Users/wejarrard/projects/tf-binding/data/jsonl_output/dataset_101.jsonl.gz.out
Downloaded inference/output/AR-22Rv1-greater-equal-2024-08-09-10-27-48/dataset_102.jsonl.gz.out to /Users/wejarrard/projects/tf-binding/data/jsonl_output/dataset_102.jsonl.gz.out
Downloaded inference/output/AR-22Rv1-greater-equal-2024-08-09-10-27-48/dataset_103.jsonl.gz.out to /Users/wejar

In [57]:
import pandas as pd
import glob
import matplotlib.pyplot as plt


# Path to the directory containing JSON files
json_dir = "/Users/wejarrard/projects/tf-binding/data/jsonl_output"

# Get a list of all JSON files in the directory
json_files = glob.glob(f"{json_dir}/*.jsonl.gz.out")

# Read each JSON file into a DataFrame and store in a list
dataframes = [pd.read_json(file) for file in json_files]

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dataframes, ignore_index=True)

df

,chr_name,start,end,cell_line,targets,predicted,weights,probabilities,enhancer,promoter
0,chr17,68474329,68474711,A549,0,0,0,0.001170,0,0
1,chr17,68474442,68474704,HEK_293,0,0,0,0.000008,0,0
2,chr17,68485145,68486170,THP-1,0,0,0,0.296090,0,0
3,chr17,68485778,68486177,A549,0,0,0,0.007806,0,0
4,chr17,68490815,68492311,A549,0,0,0,0.000170,0,0
...,...,...,...,...,...,...,...,...,...,...
602250,chr6,100080548,100080721,THP-1,0,1,0,0.970661,0,0
602251,chr6,100088028,100088349,LNCAP,1,1,6,0.994816,0,0
602252,chr6,100088048,100088425,PC-3,0,1,0,0.910881,0,0
602253,chr6,100096399,100097117,THP-1,0,0,0,0.000038,0,0


In [58]:
# get dataframe of false positives
df['tp'] = (df['targets'] == 1) & (df['predicted'] == 1)
df['fp'] = (df['targets'] == 0) & (df['predicted'] == 1)
df['fn'] = (df['targets'] == 1) & (df['predicted'] == 0)
df['tn'] = (df['targets'] == 0) & (df['predicted'] == 0)

df_fp = df[df['fp'] == True]

# get dataframe of true negatives
df_tn = df[df['tn'] == True]

# get dataframe of targets and weight = 0
df_targets = df[df['targets'] == 1]
df_fp


,chr_name,start,end,cell_line,targets,predicted,weights,probabilities,enhancer,promoter,tp,fp,fn,tn
5,chr17,68491295,68492197,THP-1,0,1,0,0.988258,0,0,False,True,False,False
16,chr17,68511548,68512913,LNCAP,0,1,3,0.700698,0,0,False,True,False,False
22,chr17,68534261,68535323,PC-3,0,1,0,0.997812,0,0,False,True,False,False
27,chr17,68542585,68543560,PC-3,0,1,0,0.984731,0,0,False,True,False,False
35,chr17,68580158,68580864,THP-1,0,1,0,0.995159,0,0,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602223,chr6,99913481,99914900,THP-1,0,1,0,0.990521,0,0,False,True,False,False
602231,chr6,99980033,99980292,PC-3,0,1,0,0.556864,0,0,False,True,False,False
602234,chr6,100000144,100000488,THP-1,0,1,0,0.633407,0,0,False,True,False,False
602250,chr6,100080548,100080721,THP-1,0,1,0,0.970661,0,0,False,True,False,False


In [59]:
# filter out weights that are >0 out of both df_fp and tf_tn
df_fp = df_fp[df_fp['weights'] == 0]
df_tn = df_tn[df_tn['weights'] == 0]


In [60]:
print( df_targets['cell_line'].unique())

['LNCAP' 'THP-1' 'PC-3' 'HEK_293' 'A549']


In [66]:
# make new dataframe that is equal parts true negatives and false positives equal to the number of targets
# make sure its balanced by cell_line


df_fp_sample = []
df_tn_sample = []
print("total positives:", len(df_targets))
for cell_line in df_targets['cell_line'].unique():
    print(cell_line)
    df_targets_cell_line = df_targets[df_targets['cell_line'] == cell_line]

    # total positives
    print("positives:", len(df_targets_cell_line))
    # total negatives
    print("false positives:", len(df_fp))
    print("true negatives:", len(df_tn))

    # amount to sample
    sample_size = len(df_targets_cell_line) // 2

    # if df_fp is less than half the sample size, sample all of it and the rest from df_tn, else sample half from each
    if len(df_fp) < sample_size:
        df_fp_sample_cell_line = df_fp
        df_tn_sample_cell_line = df_tn.sample(n=sample_size - len(df_fp))
    else:
        df_fp_sample_cell_line = df_fp.sample(n=sample_size)
        df_tn_sample_cell_line = df_tn.sample(n=sample_size)
    
    df_fp_sample.append(df_fp_sample_cell_line)
    df_tn_sample.append(df_tn_sample_cell_line)

df_fp_sample = pd.concat(df_fp_sample)
df_tn_sample = pd.concat(df_tn_sample)

df_negative_samples = pd.concat([df_fp_sample, df_tn_sample])
df_negative_samples

total positives: 50488
LNCAP
positives: 49660
false positives: 85483
true negatives: 425078
THP-1
positives: 293
false positives: 85483
true negatives: 425078
PC-3
positives: 268
false positives: 85483
true negatives: 425078
HEK_293
positives: 131
false positives: 85483
true negatives: 425078
A549
positives: 136
false positives: 85483
true negatives: 425078


,chr_name,start,end,cell_line,targets,predicted,weights,probabilities,enhancer,promoter,tp,fp,fn,tn
48189,chr12,79424136,79424698,PC-3,0,1,0,0.983021,0,0,False,True,False,False
327921,chr12,106357441,106358628,A549,0,1,0,0.881063,0,0,False,True,False,False
53213,chr2,206105370,206106972,A549,0,1,0,0.846182,0,0,False,True,False,False
134113,chrX,45479264,45480424,PC-3,0,1,0,0.996651,0,0,False,True,False,False
155935,chr5,61965198,61965373,THP-1,0,1,0,0.876553,0,0,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289045,chr4,183823438,183823716,A549,0,0,0,0.000297,0,0,False,False,False,True
240716,chr14,60197501,60198390,THP-1,0,0,0,0.010715,0,0,False,False,False,True
185840,chr4,187764235,187764684,THP-1,0,0,0,0.000002,0,0,False,False,False,True
158510,chr16,5616107,5616361,HEK_293,0,0,0,0.000003,0,0,False,False,False,True


In [67]:
targeted_training_df = pd.concat([df_negative_samples, df_targets])
targeted_training_df

,chr_name,start,end,cell_line,targets,predicted,weights,probabilities,enhancer,promoter,tp,fp,fn,tn
48189,chr12,79424136,79424698,PC-3,0,1,0,0.983021,0,0,False,True,False,False
327921,chr12,106357441,106358628,A549,0,1,0,0.881063,0,0,False,True,False,False
53213,chr2,206105370,206106972,A549,0,1,0,0.846182,0,0,False,True,False,False
134113,chrX,45479264,45480424,PC-3,0,1,0,0.996651,0,0,False,True,False,False
155935,chr5,61965198,61965373,THP-1,0,1,0,0.876553,0,0,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602166,chr6,99562815,99563367,LNCAP,1,1,26,0.997789,0,0,True,False,False,False
602169,chr6,99568217,99568971,LNCAP,1,0,8,0.143220,0,0,False,False,True,False
602197,chr6,99761756,99762133,LNCAP,1,1,8,0.959449,0,0,True,False,False,False
602230,chr6,99979898,99980356,LNCAP,1,1,31,0.998806,0,0,True,False,False,False


In [69]:
targeted_training_df = targeted_training_df[['chr_name', 'start', 'end', 'cell_line', 'targets', 'weights', 'enhancer', 'promoter']]
targeted_training_df.columns = ['chr', 'start', 'end', 'cell_line', 'label', 'count', 'region_type_enhancer', 'region_type_promoter']
targeted_training_df

,chr,start,end,cell_line,label,count,region_type_enhancer,region_type_promoter
48189,chr12,79424136,79424698,PC-3,0,0,0,0
327921,chr12,106357441,106358628,A549,0,0,0,0
53213,chr2,206105370,206106972,A549,0,0,0,0
134113,chrX,45479264,45480424,PC-3,0,0,0,0
155935,chr5,61965198,61965373,THP-1,0,0,0,0
...,...,...,...,...,...,...,...,...
602166,chr6,99562815,99563367,LNCAP,1,26,0,0
602169,chr6,99568217,99568971,LNCAP,1,8,0,0
602197,chr6,99761756,99762133,LNCAP,1,8,0,0
602230,chr6,99979898,99980356,LNCAP,1,31,0,0


In [71]:
# save as /Users/wejarrard/projects/tf-binding/data/data_splits/training_combined.csv
targeted_training_df.to_csv("/Users/wejarrard/projects/tf-binding/data/data_splits/training_combined.csv", index=False, sep="\t")